In [2]:
%matplotlib inline
import numpy as np
import json
import pandas as pd
import geopandas
import geocoder
from geopandas.tools import sjoin
from geopy.geocoders import Nominatim
from shapely.geometry import Point
from geopy.exc import GeocoderTimedOut

geolocator = Nominatim()

In [3]:
for file in ['train', 'test']:
    train_info = pd.read_json('data/backup/{}.json'.format(file))
    train = pd.read_json('data/{}.json'.format(file))

    lats = []
    longs = []
    for lat, long, address in train_info[['latitude', 'longitude', 'street_address']].values:
        if lat == 0. or long == 0.:
            while True:
                try:
                    location = geolocator.geocode(address)
                except Exception:
                    #print ('timeout, retry')
                    continue
                break
            lats.append(location.latitude)
            longs.append(location.longitude)
            print ((location.latitude, location.longitude))
        else:
            lats.append(lat)
            longs.append(long)

    train['latitude'] = lats
    train['longitude'] = longs

    train.to_json('data/{}.json'.format(file), orient='records')

(29.9748076, -82.1363356)
(40.59678, -73.9750389)
(32.7790805284282, -91.799182096111)
(35.1380985, -90.007073)
(40.454348, -79.981681)
(39.7311422, -105.021738)
(40.7743715306122, -73.9497069591837)
(39.2413453, -9.3087605)
(46.3815165, -124.0553937)
(41.6866131111111, -71.156345)
(44.922287, -69.9509319)
(39.75560005, -104.994796349865)
(42.7569651, -71.4804334)
(40.6970126122449, -73.3902216734694)
(29.5496019, -98.5332983)
(37.265754, 44.0398983)
(45.5254831342282, -122.616047)
(60.6864639, 13.7099309)
(40.5774143410926, -74.6382967155051)
(32.0441805306122, -81.107538)
(32.0498347272727, -81.1096415454546)
(40.7334326530612, -74.0071844693878)
(34.0932966, -118.302499)
(50.8235034, -0.1310935)
(25.2727794, 55.3260646)
(41.69602825, -87.6241898642143)
(46.3815165, -124.0553937)
(59.4622391428571, -135.309446653061)
(47.0277720424441, -91.6733254291717)
(39.5303075306122, -119.812594438776)
(43.184007, 21.7971241)
(40.8012571, -73.96065011003)
(50.8235034, -0.1310935)
(40.7630702, -